## Objetivo
1. Fazer scrapy de uma pagina de noticias
2. Deixa somente os conteudo relevantes
3. Armazenar os dados no BigQuery

## 0. Importando as bibliotecas

In [6]:
import pandas_gbq
from functions import *
from gcp_auth import credentials

## 1. Configurando o Webscraping

1. Primeiro, o código define duas variáveis:
    * `url`: Armazena a URL da página que será acessada, neste caso, a página da Globo.
    * `headers`: É um dicionário que contém informações sobre o navegador que está fazendo a requisição. Aqui, define o `User-Agent` como "Mozilla/5.0", simulando um navegador comum.

1. Obtendo o HTML da página:
    * A linha `response = requests.get(url, headers=headers)` usa a biblioteca requests para fazer uma requisição GET à URL especificada no url.
1. Extraindo dados com BeautifulSoup:

    * A linha `page = BeautifulSoup(response.text, 'html.parser')` usa a biblioteca BeautifulSoup para transformar o texto HTML da resposta em um objeto que pode ser facilmente navegado e analisado.
1. Selecionando a tag dos links:
    * Por fim, o código usa o método `find` do objeto `soup` para encontrar a todas as tag (`a`) na página.

In [7]:
url = "https://www.globo.com/"
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
page = BeautifulSoup(response.text, "html.parser")
    
urls = page.find_all("a")

5. Selecionando somente as tags que contem links de noticias 
* As variaveis `tag_class1` e `tag_class2` armazenam strings que representam nomes de classes em elementos HTML.
*  Essas classes são usadas para identificar elementos que contêm links para artigos de notícias utilizadas pela Globo.

In [8]:
tag_class1 = "post__title"
tag_class2 = "post-multicontent__link--title__text"

news_url_list = []
for url in urls:
    if(url.h2 != None) and (url.h2.get("class") != None):
        if tag_class1 in url.h2.get("class"):
            news_url_list.append(url.get("href"))
        elif tag_class2 in url.h2.get("class"):
            news_url_list.append(url.get("href"))

print(f'Quantidade de notícias: {len(news_url_list)}')

Quantidade de notícias: 67


## 2. Raspagem do conteúdo das notícias

Nessa parte estou utilizando uma função chamada `df_news_data`, onde os processos estão mais detalhados abaixo

Processo:

1. Loop: A função df_news_data utiliza um loop para iterar por cada URL na lista news_url_list.

1. Requisição HTTP: Para cada URL, a função faz uma requisição HTTP para recuperar o conteúdo HTML da página de notícia.

1. Análise de HTML: O HTML da página de notícia é então analisado para extrair os dados desejados:

    * url: A URL da própria notícia.
    * date: A data da publicação da notícia.
    * author: O autor da notícia (se disponível).
    * title: O título da notícia.
    * subtitle: O subtítulo da notícia (se disponível).
    * text: O corpo principal do texto da notícia.
    
1. Armazenamento de Dados: Os dados extraídos para cada notícia são armazenados em um dicionário.

1. Agregação de Dados: Todos os dicionários de notícias individuais são concatenados em um único dataframe pandas.

In [9]:
df_news = df_news_data(news_url_list)

Inicio da raspagem de dados
Erro ao fazer scraping da URL https://extra.globo.com/entretenimento/noticia/2024/05/apos-segunda-cirurgia-exame-de-tony-ramos-aponta-importante-melhora-diz-boletim-medico.ghtml: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Coleta finalizada


## 3. Salvando os dados coletados no BigQuery

In [10]:
pandas_gbq.to_gbq(df_news, 
                  "bigquery_news.news_data", 
                  project_id="project-scrapy-news",
                  if_exists="replace",
                  credentials=credentials)

100%|██████████| 1/1 [00:00<?, ?it/s]
